In [1]:
import ast
import glob
import json
import os
import time
from enum import Enum
from typing import Dict, List, Optional

import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel, Field, field_validator
from tqdm import tqdm

from scripts.report_agent import ReportAgent

load_dotenv()

True

In [2]:
# Fungsi untuk membuat direktori jika belum ada
def create_output_directory(output_path: str):
    """
    Ensure that the output directory exists.
    """
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Fungsi untuk memproses satu putaran data (first turn atau second turn)
def process_turn(query_prompt: str, schema_class: BaseModel, input_dir: str, output_dir: str, input_file: str = None, glob_path: str = "*/*.json"):
    """
    Process a single round of data extraction (first or second turn) and save the results to the specified output directory.
    
    Args:
    - query_prompt (str): The query prompt for the report extraction.
    - schema_class (BaseModel): The schema class for validating the extracted data.
    - input_dir (str): The directory containing input JSON files.
    - output_dir (str): The directory where the results will be saved.
    - input_file (str, optional): If provided, the function will filter data based on this file (e.g., the first turn results).
    - glob_path (str, optional): The glob pattern to match input files. Default is "*/*.json".
    """
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Define the list of files to process based on the glob pattern
    files = glob.glob(os.path.join(input_dir, glob_path))
    
    # Check if files exist, and raise an error if not
    if not files:
        raise FileNotFoundError(f"No files matched the glob pattern: '{glob_path}' in directory '{input_dir}'")
    
    # Process each file
    for file in tqdm(files, desc="Processing Companies"):
        try:
            company = os.path.splitext(os.path.basename(file))[0]
            agent = ReportAgent(os.environ["LLM"], schema_class, query_prompt)
            
            # Handle data processing for first or second turn
            if input_file:
                # For the second turn, filter data based on the input Excel file
                fdata = pd.read_excel(input_file)
                fdata = fdata[fdata['company'] == company]
                fdata = fdata.drop("sources", axis=1)
                
                # Loop through each row of filtered data
                for idx, row in fdata.iterrows():
                    prompt_data = row.to_dict()
                    result = agent.get_data(prompt_data)
                    result['data'].update(prompt_data)
                    
                    # Generate output path and save result
                    relative_path = os.path.relpath(file, input_dir)
                    output_path = os.path.join(output_dir, relative_path).replace(".json", f"_{idx+1}.json")
                    create_output_directory(output_path)
                    with open(output_path, 'w') as f:
                        json.dump(result, f, indent=4)
            else:
                # For the first turn, directly use the company data
                result = agent.get_data({'company': company})
                result['data']['company'] = company
                
                # Generate output path and save result
                relative_path = os.path.relpath(file, input_dir)
                output_path = os.path.join(output_dir, relative_path)
                create_output_directory(output_path)
                with open(output_path, 'w') as f:
                    json.dump(result, f, indent=4)
                    
        except Exception as e:
            print(f"\nError processing {company}: {str(e)}")
        
        # Pause to avoid overloading the API
        # time.sleep(30)
    
        # After processing all files, save results to an Excel file
        save_results_to_excel(output_dir, glob_path)

# Fungsi untuk menyimpan hasil pemrosesan ke dalam file Excel
def save_results_to_excel(output_dir: str, glob_path: str):
    """
    Save the processed results into an Excel file after extraction.
    """
    output_files = glob.glob(os.path.join(output_dir, glob_path))
    output_list = []
    
    for file in output_files:
        with open(file, 'r') as f:
            result = json.load(f)
        result['data'].update({'sources': result['sources']})
        output_list.append(result['data'])
    
    output_df = pd.DataFrame(output_list)
    output_df.to_excel(f"{output_dir}/result.xlsx", index=False)

### Start here

First Turn
- Siapkan query prompt dengan placeholder {company}
- Siapkan schema / variabel yang ingin di-crawl
- Siapkan `input_dir`, `output_dir`, dan `glob_path`

    `input_dir` dan `glob_path` saling berhubungan. Input direktori adalah tempat file json disimpan.

    Misalnya, file json terletak di path `data/JSON/AR/ID_ADRO_AR_2022.json`. 

    Maka `input_dir` bisa berupa `data`, lalu `glob_path`-nya adalah `*/*/*.json`
    ATAU jika `input_dir` berupa `data/JSON`, lalu `glob_path`-nya adalah `*/*.json`

## Phase 1

In [3]:
# FIRST TURN

query_prompt_1 = """
Extract the complete details of the Board of Directors (BOD) and the Board of Commissioners (BOC) for {company} as provided in the company's annual report.
Ensure that you retrieve both the full names and official positions for each board member. 
You may perform multiple, detailed queries with the retriever tool to verify the accuracy and completeness of the extracted information.
"""

class BoardEntry(BaseModel):
    name: str = Field(
        ...,
        description="Full name of the board member (BOD or BOC) as listed in the company's annual report.",
        example="Giovanny Alberta"
    )
    position: str = Field(
        ...,
        description="Official position or title held by the board member within the company, as reported in the annual report.",
        example="Direktur Marketing"
    )

class GovernanceBody(BaseModel):
    bod: List[BoardEntry] = Field(
        ...,
        description="A list of all Board of Directors (BOD) members, including their full names and official positions.",
        example=[{"name": "Giovanny Alberta", "position": "Direktur Marketing"}]
    )
    boc: List[BoardEntry] = Field(
        ...,
        description="A list of all Board of Commissioners (BOC) members, including their full names and official positions.",
        example=[{"name": "Giovanny Alberta", "position": "Direktur Marketing"}]
    )


# Proses first turn
process_turn(
    query_prompt=query_prompt_1,
    schema_class=GovernanceBody,
    input_dir="CG REPORT",
    output_dir="results_1st_turn",
    glob_path="*.json"
)

Processing Companies: 100%|██████████| 10/10 [03:05<00:00, 18.54s/it]


## Phase 2

- Siapkan query prompt yang memiliki placeholder sesuai variabel yang ingin dicari, dimana variabel tersebut seharusnya sudah didapatkan dari first turn.
- Masukkan parameter tambahan yaitu `input_file` yang merupakan hasil dari path excel hasil dari first turn

In [4]:
df_phase_1 = pd.read_excel('results_1st_turn/result.xlsx')

for col in df_phase_1.columns:
    if col != 'company':
        df_phase_1[col] = df_phase_1[col].apply(lambda x: ast.literal_eval(x))

postprocess = []
for idx, row in df_phase_1.iterrows():
    # Unpack bod
    for d in row.get('bod'):
        postprocess.append({
            "name": d['name'],
            "position": d['position'],
            "company": row.get('company'),
            "sources": row.get('sources')
        })
    
    # Unpack boc
    for d in row.get('boc'):
        postprocess.append({
            "name": d['name'],
            "position": d['position'],
            "company": row.get('company'),
            "sources": row.get('sources')
        })

df_postpro = pd.DataFrame(postprocess)
df_postpro.to_excel("results_1st_turn/result_postpro.xlsx", index=False)

In [5]:
class EducationEntry(BaseModel):
    major: Optional[str] = Field(
        None,
        description=(
            "Program studi atau jurusan yang ditempuh oleh individu, "
            "sesuai dengan informasi yang tercantum pada annual report perusahaan."
        ),
        example="Accounting"
    )
    university: Optional[str] = Field(
        None,
        description=(
            "Nama universitas atau institusi pendidikan tempat individu menyelesaikan studi, "
            "berdasarkan data annual report perusahaan."
        ),
        example="Universitas Airlangga"
    )
    country: Optional[str] = Field(
        None,
        description=(
            "Negara tempat institusi pendidikan tersebut berada, "
            "sesuai dengan laporan tahunan perusahaan."
        ),
        example="Indonesia"
    )

class WorkExperience(BaseModel):
    position: Optional[str] = Field(
        None,
        description=(
            "Jabatan atau posisi yang diemban oleh individu di perusahaan, "
            "mengacu pada data dalam annual report."
        ),
        example="Research Assistant"
    )
    company: Optional[str] = Field(
        None,
        description=(
            "Nama perusahaan atau institusi tempat individu bekerja, "
            "sebagaimana tercantum dalam laporan tahunan."
        ),
        example="Center for Environmental, Social, and Governance Studies (CESGS)"
    )
    country: Optional[str] = Field(
        None,
        description=(
            "Negara tempat perusahaan beroperasi, "
            "berdasarkan informasi dalam annual report."
        ),
        example="Indonesia"
    )
    start_year: Optional[int] = Field(
        None,
        description="Tahun mulai bekerja yang tercatat dalam annual report perusahaan.",
        example=2024
    )
    end_year: Optional[int] = Field(
        None,
        description=(
            "Tahun berakhir bekerja. Jika individu masih aktif bekerja, gunakan None, "
            "sesuai dengan data annual report."
        ),
        example=2026
    )

class IndividualProfile(BaseModel):
    nationality: str = Field(
        ...,
        description="Kewarganegaraan individu sebagaimana tercantum dalam annual report perusahaan.",
        example="Indonesian"
    )
    edu_s1: Optional[EducationEntry] = Field(
        None,
        description="Detail pendidikan S1, berdasarkan data annual report perusahaan.",
        example={"major": "Accounting", "university": "Universitas Airlangga", "country": "Indonesia"}
    )
    edu_s2: Optional[EducationEntry] = Field(
        None,
        description="Detail pendidikan S2, berdasarkan data annual report perusahaan.",
        example={"major": "Finance", "university": "Universitas Indonesia", "country": "Indonesia"}
    )
    edu_s3: Optional[EducationEntry] = Field(
        None,
        description="Detail pendidikan S3, berdasarkan data annual report perusahaan.",
        example={"major": "Engineering", "university": "Institut Teknologi Bandung", "country": "Indonesia"}
    )
    edu_profesi: Optional[EducationEntry] = Field(
        None,
        description="Detail pendidikan Profesi, berdasarkan data annual report perusahaan.",
        example={"major": "Legal Profession", "university": "Universitas Gadjah Mada", "country": "Indonesia"}
    )
    edu_vokasi: Optional[EducationEntry] = Field(
        None,
        description="Detail pendidikan Vokasi, berdasarkan data annual report perusahaan.",
        example={"major": "Computer Technician", "university": "Politeknik Negeri Jakarta", "country": "Indonesia"}
    )
    work_experience: List[WorkExperience] = Field(
        ...,
        description="Data pengalaman kerja individu sebagaimana tercantum dalam annual report perusahaan.",
        example=[
            {
                "position": "Research Assistant",
                "company": "CESGS",
                "country": "Indonesia",
                "start_year": 2024,
                "end_year": None
            }
        ]
    )

query_prompt_2 = """
Extract the bachelor's (S1) degree details and work experience information for {name}, who currently holds the position of {position} 
at {company}, from the company's annual report. For the bachelor's degree, retrieve comprehensive details including the major, 
university, and country of education. Additionally, extract work experience details such as the job title, company name, country, 
and the start and end years. You may perform iterative and targeted queries using the retriever tool to ensure the accuracy 
and completeness of the extracted data.
""".strip()

In [6]:
process_turn(
    query_prompt=query_prompt_2,
    schema_class=IndividualProfile,
    input_dir="CG REPORT",
    output_dir="results_2nd_turn",
    input_file="results_1st_turn/result_postpro.xlsx",
    glob_path="*.json"
)

Processing Companies: 100%|██████████| 10/10 [23:46<00:00, 142.67s/it]
